In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import time
import hashlib
import hmac
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


os.environ.pop("API_KEY")
os.environ.pop("SECRET_KEY")

load_dotenv()

API_KEY = os.environ["API_KEY"]
API_SECRET = os.environ["SECRET_KEY"]

# Binance API Helper Functions

In [3]:
def boilerplate(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = hmac.new(
        API_SECRET.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    # Add the signature to the parameters
    params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the GET request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")
    
def boilerplate1(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    # signature = hmac.new(
    #     API_SECRET.encode('utf-8'),
    #     query_string.encode('utf-8'),
    #     hashlib.sha256
    # ).hexdigest()

    # # Add the signature to the parameters
    # params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the GET request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")
    

def boilerplate_post(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = hmac.new(
        API_SECRET.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    # Add the signature to the parameters
    params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the POST request
    response = requests.post(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")

# Dual Investment Strategy Helper Functions
- **Put** Indicates that you are trying to buy the crypto product/ coin/ asset with stable coin. It means you are trying to buy Low
- **Call** Indicates that you are trying to Sell the crypto product/ coin/ asset for stable coin. It means you are trying to Sell High


In [4]:
def get_DCI_products(optionType, exercisedCoin, investCoin, pageSize, pageIndex):

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/list'

    # Define request parameters
    params = {
        'optionType': optionType,  #'CALL' or 'PUT'
        'exercisedCoin': exercisedCoin,  # Target exercised asset
        'investCoin': investCoin,  # Asset used for subscribing
        'pageSize': pageSize,  # Optional
        'pageIndex': pageIndex,  # Optional
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)


In [5]:
def get_DCI_account():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/accounts'

    # Define request parameters
    params = {
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)



In [6]:
def get_DCI_positions():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/positions'

    # Define request parameters
    params = {
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }
    return boilerplate(params, endpoint)


In [7]:
def get_price_historical(symbol, interval):
    
    # Define the endpoint and base URL
    endpoint = '/api/v3/klines'

    # Define request parameters
    params = {
        'symbol': symbol,   
        'interval': interval  
    }
    return boilerplate1(params, endpoint)

In [8]:
def get_full_Assets():

    # Define the endpoint and base URL
    endpoint = '/sapi/v3/asset/getUserAsset'

    # Define request parameters
    params = {
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate_post(params, endpoint)

In [9]:
def get_DualInvestments_summary():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/accounts'

    # Define request parameters
    params = {
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)

In [10]:
def get_price(symbol):
    # Define the endpoint and base URL
    endpoint = '/api/v3/avgPrice'

    # Define request parameters
    params = {
        'symbol': symbol
    }

    return boilerplate1(params, endpoint)

def get_all_prices():
    symbols = ["BTCUSDC", "ETHUSDC", "SOLUSDC", "BNBUSDC", "DOGEUSDC"]
    symbol_price_dict = {}
    for symbol in symbols:
        price = float(get_price(symbol)["price"])
        symbol_price_dict[symbol] = price
    return symbol_price_dict

In [11]:
def get_full_DualInvestments(settled):

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/positions'

    # Define request parameters
    params = {
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }
    # Await boilerplate function if it's an async function
    response = boilerplate(params, endpoint)
    list_of_DI = response.get("list", [])
    
    # Use filter correctly to match items with settled status
    # value = list(filter(lambda x: x["purchaseStatus"], list_of_DI))
    value = list(filter(lambda x: x["purchaseStatus"] == settled, list_of_DI))
    return value

prices = get_all_prices()

    
def get_full_DualInvestments_agg(settled):
    prices = get_all_prices()
    value = get_full_DualInvestments(settled)
    final = 0
    for item in value:
            final += float(item["subscriptionAmount"])* prices.get(f"{item['investCoin']}USDC", 1)
    return final


            

    

In [12]:
get_full_DualInvestments_agg("PURCHASE_SUCCESS")

0

In [13]:
get_DualInvestments_summary()

{'totalAmountInBTC': '0', 'totalAmountInUSDT': '0'}

# Test (required input)

In [14]:
target = "ETH"
currency = "USDC"
USDamt = 24476


if target=="BTC":
    curr_price = float(get_price("BTCUSDC")["price"]) #BTC
else:
    curr_price = float(get_price("ETHUSDC")["price"]) #ETH

print(f"{target} price {curr_price}")

Direction = "CALL" # Sell BTC/ETH High
# Direction = "PUT" # Buy BTC/ETH Low

filename = f"./figures/{datetime.now().strftime('%Y%m%d')}/fig_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{Direction}_{target+currency}.html"

os.makedirs(f"./figures/{datetime.now().strftime('%Y%m%d')}/", exist_ok=True)


if Direction=="CALL":
    AssetItem = target
    TargetItem = currency
else:
    TargetItem= target
    AssetItem = currency


ETH price 3404.96014868


# Dual Investment tracking

In [15]:
def get_DualInvestment_asset(Direction, TargetItem, AssetItem):
    print(Direction, TargetItem, AssetItem)
    first = get_DCI_products(Direction, TargetItem, AssetItem, pageIndex=1, pageSize=1)
    full_list = first["list"]
    total = first["total"]
    print(total)

    for i in range(total//100+1):
        try:
            curr = get_DCI_products(Direction, TargetItem, AssetItem, pageIndex=i+1, pageSize=100)
            full_list.extend(curr["list"])
        except Exception as e:
            print(f"ERROR found: {e}")

    print("---------------------------")
    print(len(full_list))
    df = pd.DataFrame(full_list)
    df["curr_price"]  = curr_price
    df["strikePrice"] = pd.to_numeric(df["strikePrice"])

    df["Percent_to_strikeprice"] = 100*(df["strikePrice"] - df["curr_price"])/df["curr_price"]

    df = df.sort_values(by=['apr', 'duration'], ascending=[False, True])
    df["strikePrice"] = df.strikePrice.astype(float)
    df["apr"] = df.apr.astype(float)
    df["1000return"] = df["apr"]*df["duration"]*1000/365
    df["USDamt"] = df["1000return"]*USDamt/1000
    return df

# df = get_DualInvestment_asset(Direction, TargetItem, AssetItem)

In [16]:
listt = ["USDC", "USDT", "FDUSD"]
fin = []

if Direction=="CALL":
    for i in listt:
        try:
            findf = get_DualInvestment_asset(Direction, i, AssetItem)
            fin.append(findf)
        except Exception as e:
            print(f"exception found: {e}")
else:
    for i in listt:
        findf = get_DualInvestment_asset(Direction, TargetItem, i)
        fin.append(findf)

df = pd.concat(fin)


CALL USDC ETH
574
---------------------------
575
CALL USDT ETH
538
---------------------------
539
CALL FDUSD ETH
564
---------------------------
565


In [17]:
df.head()

,id,investCoin,exercisedCoin,strikePrice,duration,settleDate,purchaseDecimal,purchaseEndTime,canPurchase,apr,...,minAmount,maxAmount,createTimestamp,optionType,isAutoCompoundEnable,autoCompoundPlanList,curr_price,Percent_to_strikeprice,1000return,USDamt
0,1597152,ETH,USDC,3425.0,7,1735891200000,8,1735833600000,True,1.2695,...,0.0001,1200,1732323329000,CALL,True,"[STANDARD, ADVANCE]",3404.960149,0.588549,24.346575,595.906778
1,1597152,ETH,USDC,3425.0,7,1735891200000,8,1735833600000,True,1.2695,...,0.0001,1200,1732323329000,CALL,True,"[STANDARD, ADVANCE]",3404.960149,0.588549,24.346575,595.906778
2,1710345,ETH,USDC,3425.0,4,1735632000000,8,1735574400000,True,1.2147,...,0.0001,1200,1734632588000,CALL,True,"[STANDARD, ADVANCE]",3404.960149,0.588549,13.311781,325.819147
3,1597153,ETH,USDC,3450.0,7,1735891200000,8,1735833600000,True,1.1826,...,0.0001,1200,1732323329000,CALL,True,"[STANDARD, ADVANCE]",3404.960149,1.322772,22.680000,555.115680
4,1709542,ETH,USDC,3450.0,4,1735632000000,8,1735574400000,True,1.0666,...,0.0001,1200,1734630791000,CALL,True,"[STANDARD, ADVANCE]",3404.960149,1.322772,11.688767,286.094264


In [18]:
df_copy = df

# df_copy = df[df["strikePrice"]>=2200]
df_copy = df_copy[df_copy["duration"]<=20]
# df_copy = df_copy[df_copy["apr"]>=0.2]

# BTC_Buy_limit = 93000
# ETH_Buy_limit = 2900
# SOL_Buy_limit = 140


# BTC_Sell_limit = 99000
# BTC_Sell_limit_2 = 108000
# ETH_Sell_limit = 2400
# SOL_Sell_limit = 140

# BTC
# df_copy = df_copy[(df_copy["strikePrice"]<=BTC_Buy_limit)]
# df_copy = df_copy[(df_copy["strikePrice"]>=BTC_Sell_limit)]
# df_copy = df_copy[(df_copy["strikePrice"]<=BTC_Sell_limit_2)]
# df_copy = df_copy[df_copy["strikePrice"]>=55000 ]

# ETH
# df_copy = df_copy[(df_copy["strikePrice"]<=ETH_Buy_limit) ]
# df_copy = df_copy[df_copy["strikePrice"]>=2150 ]


# SOL
# df_copy = df_copy[(df_copy["strikePrice"]<=SOL_Buy_limit) & (df_copy["exercisedCoin"]=="SOL")]
# df_copy = df_copy[df_copy["strikePrice"]>=2150 ]

#resultant
# df_copy = df_copy[(df_copy["strikePrice"]<=SOL_Buy_limit) & (df_copy["exercisedCoin"]=="SOL") or (df_copy["strikePrice"]<=ETH_Buy_limit) & (df_copy["exercisedCoin"]=="ETH") or (df_copy["strikePrice"]<=BTC_Buy_limit) & (df_copy["exercisedCoin"]=="BTC")]



if Direction=="CALL":
    fig = px.scatter(df_copy, x="duration", y="apr", color='exercisedCoin', hover_data=["1000return", "USDamt", "Percent_to_strikeprice", "strikePrice"])
else:
    fig = px.scatter(df_copy, x="duration", y="apr", color='investCoin', hover_data=["1000return", "USDamt", "Percent_to_strikeprice", "strikePrice"])
fig.show()

In [19]:
df_copy_time = df_copy[df_copy["duration"]==4]

if Direction=="CALL":
    fig = px.line(df_copy_time, x="strikePrice", y="apr", color='exercisedCoin',markers=True, hover_data=["1000return", "USDamt", "Percent_to_strikeprice", "strikePrice"])
else:
    fig = px.line(df_copy_time, x="strikePrice", y="apr", color='investCoin',markers=True, hover_data=["1000return", "USDamt", "Percent_to_strikeprice", "strikePrice"])

fig.write_html(filename)

fig.show()

# Prices

In [21]:
asd = get_price_historical("BTCUSDC", "1d")
print(asd)

[[1692230400000, '28698.53000000', '28751.64000000', '25002.00000000', '26624.10000000', '1706.52490000', 1692316799999, '46699081.10577560', 29024, '556.73951000', '15419699.63658260', '0'], [1692316800000, '26613.56000000', '26825.12000000', '25620.98000000', '26045.23000000', '1196.72590000', 1692403199999, '31383284.76905080', 20162, '598.85543000', '15711706.34969550', '0'], [1692403200000, '26045.24000000', '26262.81000000', '25792.39000000', '26095.44000000', '472.84884000', 1692489599999, '12303309.86480170', 8595, '228.58194000', '5945107.14433150', '0'], [1692489600000, '26095.44000000', '26296.15000000', '25975.73000000', '26190.06000000', '282.20799000', 1692575999999, '7372324.88539810', 5681, '120.49341000', '3148019.97574110', '0'], [1692576000000, '26188.52000000', '26248.59000000', '25813.87000000', '26122.69000000', '810.59238000', 1692662399999, '21107508.55204380', 9876, '406.57366000', '10585910.78120800', '0'], [1692662400000, '26122.72000000', '26137.40000000', '